In [69]:
import torch
import numpy as np
import pandas as pd
from resnet1d.resnet1d import ResNet1D
from sklearn import preprocessing
from collections import Counter

In [70]:
NUM_CLASSES = 2 # 2 or 5

In [71]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [72]:
i2l = {'N':0, 'S':1, 'V':2, 'F':3, 'Q':4} # for 5 class model

In [73]:
model = ResNet1D(
        in_channels=1, 
        base_filters=128, 
        kernel_size=16, 
        stride=2, 
        groups=8, 
        n_block=8, 
        n_classes=NUM_CLASSES,
        downsample_gap=2, 
        increasefilter_gap=4, 
        use_do=False)

In [74]:
if NUM_CLASSES == 2:
    model.load_state_dict(torch.load("model-2c.pth", weights_only=True))
elif NUM_CLASSES == 5:
    model.load_state_dict(torch.load("model-5c.pth", weights_only=True))
else:
    raise RuntimeError()
model.eval()
model = model.to(device)
torch.save(model, 'model.pkl')  # save the model

In [75]:
input = torch.rand([1, 1, 360], dtype=torch.float).to(device)
output = model(input)
print(output)
print(output.argmax())

tensor([[-0.8327,  1.4708]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor(1, device='cuda:0')


In [76]:
df_100 = pd.read_csv('100.csv') # healthy
df_208 = pd.read_csv('208.csv') # unhealthy
print(df_100['\'MLII\''])
print(df_208['\'MLII\''])

0          995
1          995
2          995
3          995
4          995
          ... 
649995    1009
649996     935
649997     889
649998     871
649999     768
Name: 'MLII', Length: 650000, dtype: int64
0         1003
1         1003
2         1003
3         1003
4         1003
          ... 
649995     979
649996     985
649997     990
649998     992
649999    1024
Name: 'MLII', Length: 650000, dtype: int64


In [77]:
def preprocess(np_input):
    np_prep = np_input[np_input.size % 360:]
    np_prep = np_prep.reshape(-1, 360)
    np_prep = preprocessing.scale(np_prep, axis=1)
    np_prep = np.expand_dims(np_prep, 1)
    return np_prep

In [78]:
def predict(np_input):
    np_prep = preprocess(np_input)
    model_input = torch.tensor(np_prep, dtype=torch.float).to(device)
    batch_size = 64
    predictions = []
    for i in range(0, len(model_input), batch_size):
        # Extract the current batch
        batch = model_input[i:i+batch_size]
        
        # Perform model inference
        output = model(batch)
        predictions.extend(output.argmax(dim=1).cpu().numpy().tolist())
    return Counter(predictions)

In [79]:
predict(df_100['\'MLII\''].to_numpy())

Counter({0: 1804, 1: 1})

In [80]:
predict(df_208['\'MLII\''].to_numpy())

Counter({0: 1186, 1: 619})